# Tagging 

In diesem Notebook beschäftigen wir uns mit *Tagging*. Beim Tagging (auch *Annotation* genannt) reichern wir Sprachdaten an, indem wir die einzelnen Bestandteile unserer Daten, etwa Wörter oder Sätze, um Zusatzinformationen wie die Wortart oder die durch den Satz ausgedrückte "Stimmung" ergänzen. Wir versehen sozusagen jedes zu annotierende Element mit einem Schildchen (engl. *tag*), auf dem die Zusatzinformation(en) festgehalten werden.

Indem wir unsere Daten taggen, können wir sie leichter auswerten. So können wir z.&nbsp;B. bei nach Wortart annotierten Daten einfach alle Funktionswörter (Artikel, Präpositionen, Konjunktionen, etc.) herausfiltern, um sie gesondert zu untersuchen (z.&nbsp;B. "Werden in Zeitungstexten die gleichen Konjunktionen verwendet wie in Social Media-Beiträgen?") oder auch, um die Daten bereinigt von diesen inhaltsarmen (Stopp-)Wörtern unter die Lupe zu nehmen.

Um Sprachdaten zu *taggen*, benötigen wir einen *Tagger*. Für die meisten Taggingarten stehen uns bei Python gleich mehrere Tagger zu Verfügung. Viele Tagger wiederum beherrschen mehrere Taggingarten. Tagging ist natürlich sprachabhängig, weshalb wir jeweils einen für unsere Daten geeigneten Tagger benutzen müssen. Unten lernen wir pro Taggingart einen besonders guten Tagger für deutschsprachige Daten kennen, z.&nbsp;T. ergänzt um Alternativen (s. auch folgende Tabelle).

Wir schauen uns in diesem Notebook sechs Arten des Taggings an:

| **Art** | **Resultat** | **Beispiel (vereinfacht)** | **Tagger (Auswahl)**
|:-:|:-|:-|:-
| **Lemmatisierung** | Grundform (sog. *Lemma*) von Wörtern, wie sie in einem Wörterbuch stehen | Wörter<sub> Wort</sub> sind<sub>sein</sub> schön<sub>schön</sub> | `HanoverTagger`, `RNNTagger`, `stanza`, `spacy`
| **Part-of-Speech-Tagging<br>(POS-Tagging)** | Wortart von Wörtern (aber mit feineren Kategorien als in der klassischen Wortartenlehre) | Wörter<sub>NN</sub> sind<sub>VVFIN</sub> schön<sub>ADJD</sub><br><sub>*(NN: Nomen, VVFIN: Finites Verb, ADJD: Adjektiv)*</sub> | `HanoverTagger`, `RNNTagger`, `stanza`, `spacy`
| **Morphologisches Tagging** | Informationen zu Flexion und grammatischen Eigenschaften von Wörtern  | Wörter<sub>Neutrum Nominativ Plural</sub> sind<sub>3. Person Plural Präsens Indikativ</sub> schön<sub>Positiv</sub> | `RNNTagger`,`stanza`, `spacy`
| **Syntaktisches Parsing** | Informationen zum Satzbau  | Ich<sub>Subjekt</sub> liebe<sub>ROOT</sub> Wörter<sub>Akkusativobjekt</sub>| `stanza`, `spacy`
| **Named Entity Recognition** | Identifikation von Eigennamen (Personen, Orte, Institutionen, etc.) | [Peter Meier]<sub>Person</sub> arbeitet beim [Bundesgerichtshof]<sub>Institution</sub> in [Karlsruhe]<sub>Ort</sub> | `stanza`, `spacy`
| **Sentiment Analysis** | "Stimmung" eines Satzes oder Texts | [Wörter sind schön]<sub>positiv</sub><br>[Peter mag seinen aktuellen Fall nicht]<sub>negativ</sub> | `germansentiment`, `stanza`

Das eigentliche Tagging ist jeweils sehr unkompliziert – einzig die richtige Syntax muss man kennen. Die Übungen in diesem Notebook umfassen deshalb neben dem Taggingschritt auch die Auswertung getaggter Daten. Dazu verwenden wir größtenteils `pandas`. Dieses Notebook bietet Dir also auch eine gute Gelegenheit, Deine Datenanalyse-Skills auszubauen.

## Allgemein

Tagger können grundsätzlich entweder regelbasiert oder statistisch operieren. **Regelbasierte Tagger** fällen ihr Urteil ("Welche Wortart ist 'Bundesgerichtshof'?") anhand von festgelegten Regeln (etwa "Großgeschriebene Wörter sind Nomen"). **Statistische Tagger** dagegen werden mit vielen Daten trainiert. Sie errechnen basierend darauf, wie wahrscheinlich ein bestimmtes Tag für ein zu taggendes Element ist und verleihen ihm – vereinfacht formuliert – dasjenige mit der höchsten Wahrscheinlichkeit. Unabhängig von der Implementierung eines Taggers, liegt ihm jeweils ein **Tagset** zugrunde. Dieses umfasst alle möglichen Tags, die "vergebbar" sind. Beim Part-of-Speech-Tagging im Deutschen ist z.&nbsp;B. das [Stuttgart-Tübingen-TagSet](https://www.ims.uni-stuttgart.de/forschung/ressourcen/lexika/germantagsets/#id-cfcbf0a7-0) (kurz *STTS*) gängig (die Tags in der Tabelle oben stammen auch daraus).

Ebenfalls unabhängig von der Implementierung eines Taggers, ist die Tatsache, dass der Output nicht *per se* korrekt ist, sondern nur die Regeln bzw. Trainingsdaten widerspiegelt. Je mehr unsere Daten von regelhafter Standardsprache bzw. den Trainingsdaten abweichen, desto weniger zuverlässig wird die darauf basierende Annotation unserer Daten ausfallen. In jedem Fall empfiehlt es sich stets, den Output von (verschiedenen) Taggern manuell zu evaluieren.

Auch in diesem Notebook gibt es einen Anwendungsfall.

***

## 🔧 Anwendungsfall: (D)ein Korpus taggen 🏷️

Diesmal hast Du die Gelegenheit, Deine eigenen Daten zu taggen. Vielleicht hast Du inspiriert vom Notebook "Web Scraping" Dein eigenes Korpus zusammengestellt? Wenn nicht, kannst Du einen beliebigen anderen Text taggen, etwa eine frühere schriftliche Arbeit von Dir. Wichtig ist einzig, dass Deine Daten in einem für Python einlesbaren Format vorliegen, idealerweise mit der Endung ".txt" (s. aber in den Zusatzübungen zum Notebook "Input und Output", wie Word-Dateien mit der Endung ".docx" bei Python eingelesen werden können). Dir ist freigestellt, mit welchen Zusatzinformationen Du Deine Daten anreicherst. Lemmatisierung und POS-Tagging sind aber meistens sinnvoll.

In den Lösungen wird ein Musteranwendungsfall bearbeitet. Konkret soll die Verteilung von POS-Tags in Behördentexten in Standardsprache versus Leichter Sprache untersucht werden. Für dieses kleine Projekt wird neben dem eigentlichen Tagging auch der vorgelagerte Datenbeschaffungsschritt (Web Scraping) sowie die nachgelagerte Analyse und Visualisierung der Ergebnisse demonstriert. Schau Dir die Lösung zu diesem Musteranwendungsfall an, auch wenn Du Dein eigenes Korpus taggst – möglicherweise erhältst Du so wertvolle Anhaltspunkte und Inspiration.

Den Anwendungsfall bearbeitest Du am Ende des Notebooks.

***

## Installation 

Bevor wir loslegen, installieren wir sämtliche in diesem Notebook verwendeten Module über die folgende Code-Zelle. Anstatt den uns bekannten Weg über die Command Line, nutzen wir die Datei "requirements.txt", die spezifiziert, welche externen Module in diesem Notebook benötigt werden (sog. *dependencies*, also Module, von denen die Ausführbarkeit dieses Notebooks *abhängt*). Bei größeren Code-Projekten gehört es dazu, eine solche Datei mitzuliefern. 

Die Ausführung der Zelle nimmt einige Zeit in Anspruch.

In [1]:
#Durch das vorangestellte Ausrufezeichen können wir Befehle an die Command Line auch innerhalb eines Notebooks ausführen.
!pip3 install -r "../3_Dateien/Tagging/requirements.txt" 

#Solltest Du nach Ausführen dieser Zelle weiter unten unerwarteterweise auf einen 'ModuleNotFoundError' stoßen,
#dann führ statt dem obigen Code den folgenden aus:
#import sys
#!{sys.executable} -m pip install -r "../3_Dateien/Tagging/requirements.txt" 

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../3_Dateien/Tagging/requirements.txt'


Scroll durch den Output und stell sicher, dass die Installation sämtlicher Module erfolgreich war.

Wenden wir uns nun den einzelnen Taggingarten zu.

## Lemmatisierung

Bei der Lemmatisierung werden nach Genus, Kasus, Tempus etc. flektierte Wortformen auf ihre Grundform bzw. ihr Lemma (Plural: *Lemmata*), wie wir sie in einem Wörterbuch finden würden, reduziert. Bei der Wortform "Häuser" wäre das Resultat etwa "Haus". Insbesondere bei stark flektierten Sprachen wie dem Deutschen ist Lemmatisierung etwa bei der Auszählung von Wörtern interessant, zumal wir eher (wenn auch nicht immer) wissen wollen, wie häufig das Verb "brauchen" in einem bestimmten Text vorkommt, als die Häufigkeiten seiner flektierten Formen separat zu erfahren ("brauchte", "bräuchtest", "gebraucht" etc.). 

In vielen Fällen ist die Reduktion von Wortform zu Lemma eindeutig. Bei Homographen (gleichgeschriebene Wörter wie "sein" [Verb] und "sein" [Possessivpronomen zu "er"]), Eigennamen ("Herr Finkenmüller"), ungebräuchlichen Komposita ("Programmierenlernen") oder trennbaren Verben ("<u>Ruf</u> mich <u>an</u>") bestehen allerdings Fehlerquellen, die es im Blick zu behalten gilt.

Zur Lemmatisierung eignet sich u.&nbsp;a. das Modul [`HanoverTagger`](https://serwiss.bib.hs-hannover.de/frontdoor/deliver/index/docId/2457/file/wartena2023-HanTa_v1.1.0.pdf), das wir in der folgenden Code-Zelle importieren und `ht` zuweisen. Anschließend wählen wir das deutsche Sprachmodell und weisen es `ht_tagger` zu:

In [2]:
from HanTa import HanoverTagger as ht

#Der 'HanoverTagger' stellt auch ein Modell für Englisch und Niederländisch zur Verfügung, vgl. https://github.com/wartaal/HanTa/tree/master.
ht_tagger = ht.HanoverTagger('morphmodel_ger.pgz') 

Das wollen wir gleich mal ausprobieren und zwar an einem längeren Text. Wir nutzen dazu das Märchen "Des Kaisers neue Kleider", das wir auch im Notebook "Reguläre Ausdrücke" bearbeiten. Zunächst lesen wir es ein:

In [3]:
with open("../3_Dateien/Des_Kaisers_neue_Kleider/Des_Kaisers_neue_Kleider.txt") as f:
    fairytale = f.read()
    
print(fairytale)

Des Kaisers neue Kleider

Vor vielen Jahren lebte einmal ein Kaiser, der so viel auf schöne neue Kleider hielt, daß er all sein Geld ausgab, um immer recht geputzt einherzugehen. Er kümmerte sich nicht um seine Soldaten und kümmerte sich auch nicht um das Theater oder Waldpartien, außer wenn er seine neuen Kleider dabei zeigen konnte. Für jede Tageszeit hatte er einen besonderen Rock, und wie man sonst von den Königen sagt: Seine Majestät befindet sich im Staatsrat, so sagte man hier: der Kaiser ist im Ankleidezimmer. In der Hauptstadt des Landes, wo er wohnte, ging es sehr lebhaft zu, und jeden Tag kamen dort viele Fremde an. So erschienen eines Tages auch zwei Betrüger, die sich für Weber ausgaben und behaupteten, sie seien imstande, den allerschönsten Stoff, den man sich nur denken könne, zu weben. Nicht allein seien schon die Farben und das Muster außergewöhnlich schön, sondern es hätten auch die Kleider, die man aus diesem Stoff verfertigte, die wunderbare Eigenschaft, daß sie für

Auf `ht_tagger` können wir nun die Methode `tag_sent` anwenden, der wir eine Liste mit Wörtern übergeben müssen. Wie wir selbst lange Texte (nach unseren eigenen Vorstellungen flexibel) tokenisieren können, haben wir im Notebook "Funktionen und Methoden" gelernt. Hier verwenden wir den Einfachheit halber die Funktion `word_tokenize` des Moduls `nltk` (für <u>N</u>atural <u>L</u>anguage <u>T</u>ool<u>k</u>it) zur Tokenisierung unseres Märchens. Die Liste mit Wörtern lassen wir direkt im Anschluss taggen:

In [4]:
import nltk
nltk.download('punkt')

fairytale_tokenized = nltk.word_tokenize(fairytale) #Tokenisierung

fairytale_output_ht = ht_tagger.tag_sent(fairytale_tokenized) #Tagging

[nltk_data] Downloading package punkt to /Users/Yannick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Werfen wir einen Blick in den Output:

In [5]:
fairytale_output_ht[0:10]

[('Des', 'der', 'ART'),
 ('Kaisers', 'Kaiser', 'NN'),
 ('neue', 'neu', 'ADJ(A)'),
 ('Kleider', 'Kleider', 'NN'),
 ('Vor', 'vor', 'APPR'),
 ('vielen', 'viel', 'PIAT'),
 ('Jahren', 'Jahr', 'NN'),
 ('lebte', 'leben', 'VV(FIN)'),
 ('einmal', 'einmal', 'ADV'),
 ('ein', 'ein', 'ART')]

Wie wir sehen, gibt uns der `HanoverTagger` für jedes Wort ein Tupel zurück, bestehend aus der ungetaggten Wortform, dem Lemma sowie ebenfalls dem POS-Tag – letzteres kriegen wir "frei Haus" mitgeliefert.

Ein Output dieser Art lässt sich wunderbar in ein DataFrame von `pandas` übertragen (vgl. Notebook "Datenanalyse"):

In [6]:
import pandas as pd

pd.options.display.max_rows = 2500 #Erhöhen der maximal angezeigten Anzahl an Zeilen auf 2500

fairytale_df_ht = pd.DataFrame(fairytale_output_ht, columns=['Wortform', 'Lemma', 'POS'])

fairytale_df_ht

,Wortform,Lemma,POS
0,Des,der,ART
1,Kaisers,Kaiser,NN
2,neue,neu,ADJ(A)
3,Kleider,Kleider,NN
4,Vor,vor,APPR
5,vielen,viel,PIAT
6,Jahren,Jahr,NN
7,lebte,leben,VV(FIN)
8,einmal,einmal,ADV
9,ein,ein,ART


Das macht nicht nur die Darstellung ansehnlicher, sondern erleichtert auch anschließende Auswertungen enorm. So können wir uns z.&nbsp;B. ganz einfach die häufigsten Lemmata oder die Frequenz eines bestimmten Lemmas ausgeben lassen.

In [7]:
print(fairytale_df_ht.Lemma.value_counts().head(), "\n") #Häufigste Lemmata mithilfe von 'value_counts' und 'head'
print(len(fairytale_df_ht[fairytale_df_ht.Lemma == "Kaiser"])) #Häufigkeit eines bestimmten Lemmas mithilfe von Filter und 'len'

Lemma
der     176
,       151
sein     62
.        59
und      48
Name: count, dtype: int64 

24


Das Lemma "Kaiser" kommt also insgesamt 24 Mal vor. Und als Wortform?

In [8]:
len(fairytale_df_ht[fairytale_df_ht.Wortform == "Kaiser"])

20

Wenn wir also am Lemma "Kaiser" – oder anders ausgedrückt: am Konzept ᴋᴀɪsᴇʀ – interessiert sind, zeigt sich an diesem Beispiel der Nutzen von Lemmatisierung. Und wie wir gesehen haben, liefert uns der `HanoverTagger` diese Lemmata auf unkomplizierte Weise. 

*** 

✏️ **Übung 1:** Lemmatisier den Koalitionsvertrag von 2018, der sich im Ordner "3_Dateien/Koalitionsvertraege" befindet. Find dann erstens heraus, welches Lemma am häufigsten darin vorkommt sowie wie oft. Ermittle zweitens, welchen Wortformen dieses häufigste Lemma wie oft entspricht. Da der Koalitionsvertrag recht lang ist, dauert die Ausführung des Codes vielleicht etwas länger.

In [9]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

State of the Art für deutschsprachige Lemmatisierung ist eigentlich der `RNNTagger` (bzw. sein Vorgänger, der `TreeTagger`). Diesen gibt es jedoch nicht als Modul für Python, sondern nur als Command Line-Tool. Weiter ist seine Installation unter Windows zwar möglich, aber kompliziert. Der folgende Abschnitt zum `RNNTagger` richtet sich nur an Nutzende mit macOS oder Linux. 

Wenn Windows Dein Betriebssystem ist, bringst Du den `RNNTagger` am einfachsten zum Laufen, indem Du Linux als zweites Betriebssystem (konkreter: als sogenanntes *Subsystem* in Windows) auf Deinem Rechner installierst. Das ist gar nicht so kompliziert, wie es klingt. Eine Anleitung dazu findest Du im Ordner "5_Bonusmaterial" im Notebook "Linux auf Windows". Dort steht auch, wie Du den `RNNTagger` über Dein neues Subsystem ausführst. Schau in dieses Notebook oder mach direkt hier bei [Part-of-Speech-Tagging](#Part-of-Speech-Tagging) weiter.

### Lemmatisierung mit dem `RNNTagger` für macOS/Linux

Installier den `RNNTagger` auf einem Rechner mit macOS oder Linux wie folgt:

1. Lad den `RNNTagger` über [diesen Link](https://www.cis.lmu.de/~schmid/tools/RNNTagger/data/RNNTagger-1.4.7.zip) (oder von [dieser Webseite](https://www.cis.lmu.de/~schmid/tools/RNNTagger/)) herunter. Die Datei ist fast 4 GB groß, stell also sicher, dass Du eine stabile Internetverbindung sowie ausreichend Speicherplatz auf Deinem Rechner hast.
2. Entpack die heruntergeladene Datei und speicher sie im bereits existierenden Ordner "3_Dateien/RNNTagger". Stell sicher, dass die Ordnerstruktur am Ende so ausschaut:

    <img src="../3_Dateien/Grafiken_und_Videos/Ordnerstruktur_RNNTagger.png" width="800">

Wie gesagt: Der `RNNTagger` ist ein Tool für die Command Line. Wie wir aber beim Installieren der benötigten Module oben gesehen haben, können wir die Command Line auch aus einem Jupyter Notebook heraus bedienen. Das wollen wir gleich tun. 

Zunächst überprüfen wir, ob die Installation geklappt hat: Dazu müssen wir das Arbeitsverzeichnis in den Ordner verlegen, in dem sich der `RNNTagger` nun entpackt befindet.

In [10]:
"""Diese Zelle nur einmal ausführen, sonst erhältst Du eine Fehlermeldung, da relativ vom eben festgelegten
Arbeitsverzeichnis '../3_Dateien/RNNTagger' kein Verzeichnis mit dem angegebenen Pfad existiert. """
%cd ../3_Dateien/RNNTagger 

[Errno 2] No such file or directory: '../3_Dateien/RNNTagger'
/Users/Yannick/Documents/Repositories/yfrommherz.ch/website/programming/1_Notebooks


Nun legen wir im Ordner "3_Dateien/Output" eine Datei namens "test.txt" an, die wir anschließend testweise taggen. Wir beschreiben die Datei mit "Schauen wir, ob das klappt!". `echo` ist der entsprechende Command Line-Befehl für diesen Vorgang. 

In [11]:
"""Der Pfad zur neuen Datei ist relativ vom eben festgelegten Arbeitsverzeichnis aus 
(dies gilt auch für die folgenden relativen Pfade in diesem Abschnitt)."""
!echo "Schauen wir, ob das klappt!" > ../Output/test.txt 

zsh:1: no such file or directory: ../Output/test.txt


Abschließend rufen wir das Command Line-Tool `rnn-tagger-german.sh` (das sich im Unterordner "cmd" befindet) auf, und beauftragen es, den Text aus der eben geschaffenen Testdatei zu taggen.

In [12]:
!cmd/rnn-tagger-german.sh ../Output/test.txt 

zsh:1: no such file or directory: cmd/rnn-tagger-german.sh


Ähnlich wie der `HanoverTagger` beherrscht der `RNNTagger` nicht nur Lemmatisierung (drittes Element je Zeile), sondern auch morphologisches Tagging (zweites Element). In den morphologischen Tags sind wiederum auch POS-Tags enthalten (jeweils am Anfang).

Nun wollen wir ebenfalls "Des Kaisers neue Kleider" taggen, um anschließend den Output des `RNNTagger` mit demjenigen des `HanoverTagger` zu vergleichen. Anstatt der Testdatei übergeben wir dem Command Line-Tool ganz einfach das Märchen. Wir ergänzen den Befehl um `> ...`, sodass der Output direkt in eine neue Datei geschrieben wird.

In [13]:
!cmd/rnn-tagger-german.sh ../Des_Kaisers_neue_Kleider/Des_Kaisers_neue_Kleider.txt > ../Output/Des_Kaisers_neue_Kleider_output.txt

zsh:1: no such file or directory: ../Output/Des_Kaisers_neue_Kleider_output.txt


Die getaggte Datei lesen wir wiederum am besten mit `pandas` ein. Durch einen Blick in die Datei erfahren wir, dass die einzelnen Werte jeder Zeile, die wir in Spalten überführen wollen, durch "\t" voneinander abgetrennt sind. Entsprechend spezifizieren wir den `sep`-Parameter. Weiter benennen wir die Spalten mithilfe des `names`-Parameters:

In [14]:
#'names' beim Einlesen von Dateien entspricht 'columns' beim Erstellen eines DataFrames basierend auf einem Objekt im Arbeitsspeicher, s. o.
fairytale_df_rnn = pd.read_csv("../Output/Des_Kaisers_neue_Kleider_output.txt", sep="\t", names=['Wortform', 'Morphologie/POS', 'Lemma'])
fairytale_df_rnn

FileNotFoundError: [Errno 2] No such file or directory: '../Output/Des_Kaisers_neue_Kleider_output.txt'

Wie gesagt, wir wollen den Output des `RNNTagger` mit demjenigen des `HanoverTagger` vergleichen. Leider können wir nicht einfach die Spalte "Lemma" in den beiden DataFrames miteinander vergleichen, denn die im `RNNTagger` integrierte Tokenisierung weicht von derjenigen von `nltk` ab, wie wir sehen, wenn wir uns die Länge der beiden DataFrames ausgeben lassen:

In [15]:
len(fairytale_df_ht), len(fairytale_df_rnn)

NameError: name 'fairytale_df_rnn' is not defined

Offensichtlich führte die Tokenisierung des `RNNTagger` zu ein paar mehr Wörtern als diejenige von `nltk`, z.&nbsp;B. aufgrund unterschiedlicher Handhabung von Interpunktion. Dieser Umstand erschwert den zeilenweisen Vergleich der Werte in der Spalte "Lemma". 

Wir lösen dieses Problem, indem wir die Lemmatisierung durch den `HanoverTagger` ganz einfach auf Basis der Tokenisierung des `RNNTagger` (statt `nltk`) noch einmal vornehmen. Das Resultat der Tokenisierung vom `RNNTagger` liegt uns ja in der Spalte "Wortform" in `fairytale_df_rnn` vor. Der Methode `tag_sent`  des `HanoverTagger` übergeben wir ganz einfach die in eine Liste konvertierte Spalte "Wortform":

In [16]:
rnn_tokenized_ht_output = ht_tagger.tag_sent(fairytale_df_rnn.Wortform.to_list())

NameError: name 'fairytale_df_rnn' is not defined

Da wir nur an den Lemmata interessiert sind, isolieren wir diese aus den Tupeln in `rnn_tokenized_ht_output` mithilfe einer List Comprehension:

In [17]:
rnn_tokenized_ht_output = [element[1] for element in rnn_tokenized_ht_output]

NameError: name 'rnn_tokenized_ht_output' is not defined

Nun führen wir den Output der beiden Tagger zusammen, indem wir ein neues DataFrame mit den Wortformen und zwei Spalten für die beiden Lemmatisierungen schaffen:

In [18]:
fairytale_comparison = pd.DataFrame({"Wortform": fairytale_df_rnn.Wortform, 
                                     "Lemma_RNN": fairytale_df_rnn.Lemma, 
                                     "Lemma_ht": rnn_tokenized_ht_output})

NameError: name 'fairytale_df_rnn' is not defined

Schauen wir uns `fairytale_comparison` an: 

In [19]:
fairytale_comparison

NameError: name 'fairytale_comparison' is not defined

Beim Scrollen durch `fairytale_comparison` zeigt sich, dass die beiden Tagger meistens gleich lemmatisiert haben. In einigen Fällen resultierten jedoch unterschiedliche Lemmata. Folgender Code schneidet `fairytale_comparison` hinsichtlich unterschiedlicher Lemmatisierung zu und zählt deren Vorkommen auch gleich aus:

In [20]:
fairytale_comparison[fairytale_comparison.Lemma_RNN != fairytale_comparison.Lemma_ht].value_counts()

NameError: name 'fairytale_comparison' is not defined

In einigen dieser Fälle hat eindeutig einer der beiden Tagger recht (z.&nbsp;B. ist "gestehen" keine flektierte Form von "stehen", weswegen hier das Lemma "gestehen" des `RNNTagger` korrekt ist), in anderen müsste man sich den Kontext genauer anschauen (z.&nbsp;B. kann "gefällt" eine flektierte Form sowohl von "gefallen" als auch von "fällen" sein). In den meisten Fällen jedoch gibt es kein Richtig oder Falsch. So ist es schlicht Konvention, ob "solche" auf "solcher" oder "solch" reduziert werden soll. Solange dies konsistent gemacht wird, spielt es keine Rolle, welcher Konvention gefolgt wird. Schließlich garantiert die Konsistenz, dass wir zuverlässig Lemmata auszählen können.

`rnn-tagger-german` fürs Deutsche ist übrigens bei Weitem nicht der einzige Tagger, der sich im Ordner "cmd" befindet. Den `RNNTagger` gibt es für so verschiedene Sprachen wie Arabisch, Isländisch, Koreanisch, Frühneuhochdeutsch, Schweizerdeutsch oder Obersorbisch... Um Daten in einer der anderen Sprachen zu taggen, musst Du einzig den Pfad zum Tagger ersetzen.

Bevor wir mit Part-of-Speech-Tagging weitermachen, müssen wir das aktuelle Arbeitsverzeichnis zurücksetzen:

In [21]:
#Diese Zelle nur einmal ausführen, s. Begründung oben
%cd ../../1_Notebooks 

[Errno 2] No such file or directory: '../../1_Notebooks'
/Users/Yannick/Documents/Repositories/yfrommherz.ch/website/programming/1_Notebooks


## Part-of-Speech-Tagging

Beim Part-of-Speech-Tagging (kurz: *POS-Tagging*) wird die Wortart eines Worts ermittelt. POS-Tags sind aber wesentlich feiner gegliedert als die herkömmlichen paar Wortarten, die wir in der Schule gelernt haben (u.&nbsp;a. Nomen, Verben, Adjektive etc.). So wird etwa bei den Verben danach unterschieden, ob sie [finit](https://de.wikipedia.org/wiki/Finite_Verbform) sind sowie, ob es sich um ein Hilfs- oder Modalverb handelt. Wie bereits erwähnt ist fürs Deutsche das [Stuttgart-Tübingen-TagSet](https://www.ims.uni-stuttgart.de/forschung/ressourcen/lexika/germantagsets/#id-cfcbf0a7-0) der Standard (die hier nicht behandelte, aber ebenfalls beliebte korpuslinguistische Software Sketch Engine verwendet jedoch ein eigenes [Tagset](https://www.cis.uni-muenchen.de/~schmid/papers/Schmid-Laws.pdf)). Das STTS umfasst insgesamt 54 Tags. Wie auch bei der Lemmatisierung, ist die Zuordnung eines POS-Tags nicht immer eindeutig.

Zum POS-Taggen können wir, wie oben bereits gemacht, den `HanoverTagger` (sowie ebenfalls den `RNNTagger`) einsetzen. Spielen wir das noch an einem weiteren Text durch, z.&nbsp;B. dem ersten Kapitel von Niels Holgersen. Bevor wir den Text taggen können, müssen wir ihn für den `HanoverTagger` tokenisieren:

In [22]:
with open("../3_Dateien/Niels_Holgersen/Kapitel_1.txt", encoding="utf8") as f:
    niels_holgersen = f.read()
    
niels_holgersen_tokenized = nltk.word_tokenize(niels_holgersen) #Tokenisierung

niels_holgersen_output = ht_tagger.tag_sent(niels_holgersen_tokenized) #Tagging

Schauen wir uns das Ergebnis wiederum in einem DataFrame an:

In [23]:
niels_holgersen_df = pd.DataFrame(niels_holgersen_output, columns=['Wortform', 'Lemma', 'POS'])

niels_holgersen_df

,Wortform,Lemma,POS
0,I,I,NE
1,.,.,$.
2,Der,der,ART
3,Junge,Junge,NN
4,Der,der,ART
...,...,...,...
7283,sich,sich,PRF
7284,nur,nur,ADV
7285,denken,denken,VV(INF)
7286,konnte,können,VM(FIN)


Das schaut doch wunderbar aus!

***


✏️ **Übung 2:** Bring in Erfahrung, welche fünf Adjektive am häufigsten im ersten Kapitel von Niels Holgersen vorkommen.

<details>
  <summary>💡 Tipp 1</summary>
  <br>Schau in der <a href="https://www.ims.uni-stuttgart.de/forschung/ressourcen/lexika/germantagsets/#id-cfcbf0a7-0">Dokumentation des POS-Tagsets</a> nach, welche Tags für Adjektive verwendet werden. Setzt der <code>HanoverTagger</code> das Tagset eins-zu-eins um?
</details>
<br>

<details>
  <summary>💡 Tipp 2</summary>
  <br>Überleg Dir, ob Wortformen oder Lemmata besser zur Ausgabe der fünf häufigsten Adjektive geeignet sind.
</details>

In [24]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Sehr gut. Machen wir mit morphologischem Tagging weiter.

## Morphologisches Tagging

Während wir bei der Lemmatisierung flektierte Wortformen auf ihre unflektierte Grundform reduzieren, geht es beim morphologischen Tagging umgekehrt darum, herauszufinden, *wie* eine Wortform flektiert ist sowie welche grammatikalischen Merkmale sie aufweist. Die Wortform "Häuser" in einem Satz wie "Sie verkauft Häuser" liegt etwa als *neutrales* (Genus) Nomen im *Akkusativ* (Kasus) und *Plural* (Numerus) vor. Morphologische Tags werden üblicherweise kombiniert mit POS-Tags benutzt, z.&nbsp;B. um alle Verbformen in der dritten Person Singular zu extrahieren. 

Wenn Du den `RNNTagger` installiert hast, hast Du bereits gesehen, wie Du morphologische Tags (inkl. POS-Tags) erhältst. Weiter bieten die Module `spacy` und `stanza` die Möglichkeit, Texte morphologisch zu taggen. In den folgenden Zellen tun wir Folgendes:

1. Wir bereiten das morphologische Tagging mithilfe der beiden Tagger vor. 
2. Wir taggen wieder das erste Kapitel aus Niels Holgersen, wobei wir jeweils messen, wie lange das Tagging dauert. 
3. Wir überführen den Output in zwei verschiedene DataFrames.

Lies die Kommentare in den Code-Zellen aufmerksam durch, um nachzuvollziehen, wie `spacy` und `stanza` zum morphologischen Tagging eingesetzt werden.

In [25]:
#Erstens: Vorbereitung für 'spacy'
import spacy, spacy.cli

"""Download des deutschsprachigen Modells, das zum Taggen benötigt wird. Das Modell muss
nur einmal heruntergeladen werden, danach befindet es sich lokal gespeichert."""
spacy.cli.download("de_core_news_sm") 

#Initialisieren von 'spacy_tagger' mit dem deutschsprachigen Modell (anstatt 'spacy_tagger' wird auch oft die Variable 'nlp' benutzt)
spacy_tagger = spacy.load("de_core_news_sm") #'spacy' stellt weitere Modelle für Deutsch und andere Sprachen zur Verfügung

#Vorbereitung für 'stanza'
import stanza

"""Initialisieren von 'stanza_tagger' mit deutschsprachigem Modell sowie den benötigten Modellen bzw. Taggern: 
Morphologisches Tagging ist bei 'stanza' Teil von 'pos'; 'tokenize' und 'mwt' (Multiworttokenisierung) sind 
wiederum Voraussetzungen für 'pos'. Modelle werden automatisch heruntergeladen."""
stanza_tagger = stanza.Pipeline(lang="de", processors="tokenize,mwt,pos") 

     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/14.6 MB 22.5 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━ 11.4/14.6 MB 35.3 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 29.6 MB/s eta 0:00:00


✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


2024-06-11 09:48:09 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


...

2024-06-11 09:48:10 INFO: Using device: cpu


2024-06-11 09:48:10 INFO: Loading: tokenize


2024-06-11 09:48:10 INFO: Loading: mwt


2024-06-11 09:48:10 INFO: Loading: pos


2024-06-11 09:48:11 INFO: Done loading processors!


In [26]:
#Zweitens: Eigentliches Tagging inkl. Zeitmessung
import time

#Mit 'spacy'
start_time = time.time()
spacy_output = spacy_tagger(niels_holgersen)
spacy_time = time.time() - start_time

#Mit 'stanza'
start_time = time.time()
stanza_output = stanza_tagger(niels_holgersen)
stanza_time = time.time() - start_time

In [27]:
#Drittens: Überführen in DataFrame

#Output von 'spacy'
"""Das Überführen der getaggten Daten in ein DataFrame kann verschiedentlich umgesetzt werden, 
hier folgt eine besonders effiziente Variante: Wir erstellen erst eine Liste ('list_of_all_dicts'),
an die wir unten bei der Iteration über 'spacy_output' die einzelnen Zeilenwerte (Wortform und Morphologie) 
in Form eines dictionary ('dict_per_word') anhängen. Aus dem dictionary konstruieren wir anschließend
ein DataFrame."""
list_of_all_dicts = []

"""Iteration über 'spacy_output', um Wortform (über 'text'-Attribut) und morphologisches Tag ('morph') für jede 
Zeile (im finalen DataFrame) in 'dict_per_word' zu übertragen und 'list_of_all_dicts' anzuhängen. Die morphologischen Tags 
casten wir dabei in einen string und bereinigen sie von Klammern. Das Casting ermöglicht nicht nur die Bereinigung, 
sondern stellt auch sicher, dass die Tags im finalen DataFrame auch wirklich als strings vorliegen und nicht als 
spacy-eigener Datentyp. Dies ist für die Auswertung der Daten mit 'pandas'-string-Methoden entscheidend."""
for word in spacy_output:
    dict_per_word = {
        "Wortform": word.text,
        "Morphologie": str(word.morph).strip("()")}
    list_of_all_dicts.append(dict_per_word)
    
niels_holgersen_df_spacy = pd.DataFrame(list_of_all_dicts) #Erstellen eines DataFrame auf Basis der Liste mit dictionaries

#Output von 'stanza'
"""Gleiche Logik wie oben, allerdings teilt 'stanza' den Text beim Tagging zunächst in Sätze auf, weswegen wir erst über 
die Sätze (mithilfe des Attributs 'sentences') iterieren müssen, dann über die darin befindlichen Wörter ('words'), 
um schließlich auf die Wortform ('text') sowie die morphologischen Informationen ('feats') zuzugreifen."""
list_of_all_dicts = []

for sentence in stanza_output.sentences:
    for word in sentence.words:
        dict_per_word = {
            "Wortform": word.text,
            "Morphologie": word.feats}
        list_of_all_dicts.append(dict_per_word)
    
niels_holgersen_df_stanza = pd.DataFrame(list_of_all_dicts)

Werfen wir in den nächsten beiden Zellen einen Blick in die DataFrames mit den morphologischen Tags, konkret in den Schluss des Textes:

In [28]:
niels_holgersen_df_spacy.tail(20)

,Wortform,Morphologie
7418,als,
7419,flöge,Mood=Sub|Number=Sing|Person=3|Tense=Pres|VerbF...
7420,man,Case=Nom|Number=Sing|PronType=Ind
7421,fort,
7422,von,
7423,allem,Case=Dat|Gender=Neut|Number=Sing|PronType=Ind
7424,Leid,Case=Dat|Gender=Neut|Number=Sing
7425,und,
7426,allen,Case=Dat|Gender=Fem|Number=Plur|PronType=Ind
7427,Sorgen,Case=Dat|Gender=Fem|Number=Plur


In [29]:
niels_holgersen_df_stanza.tail(20)

,Wortform,Morphologie
7355,als,None
7356,flöge,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...
7357,man,Case=Nom|Number=Sing|PronType=Ind
7358,fort,None
7359,von,None
7360,allem,Case=Dat|Gender=Neut|Number=Sing|PronType=Tot
7361,Leid,Case=Dat|Gender=Neut|Number=Sing
7362,und,None
7363,allen,Case=Dat|Gender=Fem|Number=Plur|PronType=Tot
7364,Sorgen,Case=Dat|Gender=Fem|Number=Plur


Wenngleich sich der Output der beiden Tagger ähnelt, zeigen sich doch Unterschiede, etwa beim Wort "Verdrießlichkeiten", bei dem nur `stanza` das korrekte Genus taggt. Bei "flöge" hingegen liegt `spacy` in Bezug auf den Modus Konjuktiv (*<u>sub</u>junctive Mood* auf Englisch) richtig. Ein kompletter, zeilenweiser Vergleich aller Tags ist an dieser Stelle schwierig, da die beiden Tagger unterschiedlich tokenisiert haben, was zu einer abweichenden Gesamtzahl an Zeilen führt (vgl. auch Abschnitt zum `RNNTagger`). Gemeinhin gilt der Output von `stanza` als zuverlässiger als derjenige von `spacy`. Übrigens kann es sein, dass sich die morphologischen Tags in Deinem Output von den hier erwähnten unterscheiden, was darauf zurückzuführen wäre, dass die zugrundeliegenden Sprachmodelle von `spacy` bzw. `stanza` aktualisiert wurden. Diese Einschränkung gilt auch bei den übrigen Taggingarten.

Abgesehen von der Qualität der Tags könnten sich die beiden Tagger (auch) mit Blick auf die Effizienz unterscheiden. Schauen wir, wie lange sie jeweils zum Taggen des gegebenen Texts benötigt haben:

In [30]:
spacy_time, stanza_time

(2.2411839962005615, 11.668249130249023)

Hier hat `spacy` klar die Nase vorn. Die Zeiten mögen leicht variieren, `spacy` ist aber immer um ein Vielfaches schneller als `stanza`. 

Je nach Daten und Forschungsinteresse müssen wir folglich zwischen der Zuverlässigkeit der Tags – die es manuell zu beurteilen gilt – und der Effizienz des Taggings abwägen.

***

✏️ **Übung 3:** Reicher den Koalitionsvertrag von 2021 nicht nur mit morphologischen Tags, sondern auch mit Lemmata und POS-Tags an. Wähl selbst, welchen Tagger Du dazu verwendest und informier Dich ggf. in der entsprechenden Dokumentation zu den Taggingmöglichkeiten. Überführ die getaggten Daten abschließend in ein DataFrame. 

<details>
  <summary>💡 Tipp 1</summary>
  <br>Da es sich um einen langen Text handelt, bietet sich die Nutzung des effizienteren Moduls, <code>spacy</code>, an. Alternativ könntest Du auch mit dem <code>RNNTagger</code> arbeiten.
</details>
<br>

<details>
  <summary>💡 Tipp 2</summary>
  <br>Stell sicher, dass Du die morphologischen Tags als strings in das DataFrame überführst (vgl. Kommentar in der Code-Zelle "#Drittens: Überführen in DataFrame" oben). Dies ist wichtig für die anschließende Datenauswertung in Übung 4.
</details>

In [31]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

✏️ **Übung 4:** Ermittle die 20 häufigsten femininen Nomina im Koalitionsvertrag von 2021.

<details>
  <summary>💡 Tipp 1</summary>
  <br>Arbeite mit dem in Übung 3 erstellten DataFrame sowie string-Methoden (vgl. Notebook "Datenanalyse") und überleg Dir zunächst, wie Du das DataFrame filtern kannst, um nur Nomina zu erhalten, und nur solche, die auch <i>feminin</i> sind. 
</details>
<br>
<details>
  <summary>💡 Tipp 2</summary>
  <br>Mehrere Filter kannst Du bei <code>pandas</code> durch den <code>&</code>-Operator verbinden. Setz aber die einzelnen Filter in runde Klammern.
</details>
<br>
<details>
  <summary>💡 Tipp 3</summary>
  <br>Nachdem Du das DataFrame gefiltert hast, musst Du Dir überlegen, in welcher Spalte Du sinnvollerweise das Vorkommen der Werte auszählst (vgl. auch Übung 2 oben).
</details>

In [32]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Weiter geht's mit syntaktischem Parsing

## Syntaktisches Parsing

Beim syntaktischen Parsing (auch *dependency parsing* genannt) interessiert uns, welche syntaktischen Beziehungen zwischen den einzelnen Wörtern eines Satzes bestehen. Um beim minimalistischen Beispiel "Sie verkauft Häuser" zu bleiben: "Sie" entspricht in diesem Beispiel dem Subjekt, "verkauft" dem Prädikat und "Häuser" dem (Akkusativ-)objekt. Das Prädikat wird als Wurzel (engl. *root*) des Satzes bezeichnet.

Auch für syntaktisches Parsing bieten sich sowohl `spacy` als auch `stanza` an. Im Falle von `spacy` können wir bereits den oben initialisierten `spacy_tagger` benutzen, denn `spacy` taggt Sprachdaten – wie wir auch in Übung 3 gesehen haben – *standardmäßig* in Bezug auf Lemmata, POS, Morphologie und eben Syntax (sowie Named Entity Recognition, s.&nbsp;u.). Über ein [`disable`-Argument](https://spacy.io/usage/linguistic-features#disabling) im `load`-Befehl (s.&nbsp;o.) könnten wir allerdings einzelne Tagger ausschalten, was vor allem bei großen Datenmengen Sinn macht. Für `stanza` hingegen müssen wir den `stanza_tagger` modifizieren, um Daten syntaktisch taggen zu können:

In [33]:
#Die Tagger vor 'depparse' sind Abhängigkeiten desselben und müssen entsprechend mitgeladen werden
stanza_tagger = stanza.Pipeline(lang="de", processors="tokenize,mwt,pos,lemma,depparse")

2024-06-11 09:48:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-11 09:48:25 INFO: Downloaded file to /Users/Yannick/stanza_resources/resources.json


2024-06-11 09:48:26 INFO: Loading these models for language: de (German):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| pos       | gsd_charlm   |
| lemma     | gsd_nocharlm |
| depparse  | gsd_charlm   |



2024-06-11 09:48:26 INFO: Using device: cpu


2024-06-11 09:48:26 INFO: Loading: tokenize


2024-06-11 09:48:26 INFO: Loading: mwt


2024-06-11 09:48:26 INFO: Loading: pos


2024-06-11 09:48:26 INFO: Loading: lemma


2024-06-11 09:48:26 INFO: Loading: depparse


2024-06-11 09:48:27 INFO: Done loading processors!


Lassen wir uns unseren leicht ergänzten Beispielsatz in `sentence` syntaktisch auseinandernehmen, und zwar von beiden Taggern:

In [34]:
sentence = "Sie verkauft dem jungen Herrn schöne Häuser"

#Tagging
spacy_output = spacy_tagger(sentence)
stanza_output = stanza_tagger(sentence)

#Ausgabe der beiden Outputs nebeneinander mithilfe einer 'range'-Schleife
print(f"{'Wortform':10}{'spacy':10}{'stanza':10}\n{'-'*30}")
for i in range(len(spacy_output)):
    """Zugriff auf jeweilige Wortform und syntaktische Tags (bei 'spacy' mit 'dep_'-Attribut, bei 'stanza' 
    mit 'deprel'-Attribut). Da 'stanza' den zu taggenden Text jeweils erst in Sätze splittet (vgl. Kommentar oben) müssen
    wir zusätzlich den ersten (und einzigen) Satz mit dem Index null indizieren. Schöne Formatierung mit f-strings."""
    print(f"{spacy_output[i].text:10}{spacy_output[i].dep_:10}{stanza_output.sentences[0].words[i].deprel:10}")

Wortform  spacy     stanza    
------------------------------
Sie       sb        nsubj     
verkauft  ROOT      root      
dem       nk        det       
jungen    nk        amod      
Herrn     da        obj       
schöne    nk        amod      
Häuser    oa        obj       


Die beiden Tagger verwenden offenkundig unterschiedliche Tagsets. `stanza` verwendet die Tags von [Universal Dependencies](https://universaldependencies.org/de/dep/index.html), einem Projekt, das sich um einheitliche sprachübergreifende Annotation bemüht. `spacy` richtet sich auch danach, nennt die Tags aber anders. Sämtliche Tags bei `spacy` und ihre Bedeutung können wir so erfahren:

In [35]:
for label in spacy_tagger.get_pipe("parser").labels:
    print(f"{label:10}{spacy.explain(label)}")

ROOT      root
ac        adpositional case marker
adc       adjective component
ag        genitive attribute
ams       measure argument of adjective
app       apposition
avc       adverbial phrase component
cc        coordinating conjunction
cd        coordinating conjunction
cj        conjunct
cm        comparative conjunction
cp        complementizer
cvc       collocational verb construction
da        dative
dep       unclassified dependent
dm        discourse marker
ep        expletive es
ju        junctor
mnr       postnominal modifier
mo        modifier
ng        negation
nk        noun kernel element
nmc       numerical component
oa        accusative object
oc        clausal object
og        genitive object
op        prepositional object
par       parenthetical element
pd        predicate
pg        phrasal genitive
ph        placeholder
pm        morphological particle
pnc       proper noun component
punct     punctuation
rc        relative clause
re        repeated element
rs   

Im Beispiel oben unterscheidet `stanza` im Gegensatz zu `spacy` zwischen Adjektiv ("amod") und Artikel ("det"). `spacy` taggt hingegen schlicht "noun kernel element", was im Sinne von Bestandteilen der [Nominalphrase](https://de.wikipedia.org/wiki/Nominalphrase) zu verstehen ist. Andererseits punktet `spacy` bei der korrekten Kasuszuordnung der Objekte ("da" vs. "oa", also Dativ vs. Akkusativ). `stanza` auf der anderen Seite taggt in beiden Fällen "obj", was gemäß dem [Universal Dependencies-Tagset](https://universaldependencies.org/de/dep/obj.html) für das direkte Objekt, also das Akkusativobjekt, steht und insofern bei "Herrn" falsch ist. Auch hier gilt: Je nach Daten und Forschungsinteresse muss evaluiert werden, welcher Tagger die besten Ergebnisse liefert.

`spacy` implementiert übrigens die hilfreiche Möglichkeit, sich die syntaktischen Beziehungen in einem Satz visualisieren zu lassen:

In [36]:
from spacy import displacy

sentence = "Sie verkauft dem jungen Herrn schöne Häuser"

spacy_output = spacy_tagger(sentence)
displacy.render(spacy_output, style="dep", jupyter=True)

***

✏️ **Übung 5:** Find heraus, welche Wörter besonders häufig als Subjekt im Koalitionsvertrag von 2021 stehen und lass Dir die zehn Spitzenreiter ausgeben.

<details>
  <summary>💡 Tipp</summary>
  <br>Musst Du den Koalitionsvertrag dafür noch einmal taggen?
</details>

In [37]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Sehr gut. Wenden wir uns der Named Entity Recognition zu.

## Named Entity Recognition

Bei der Named Entity Recognition (abgekürzt: *NER*) geht es darum, Eigennamen, etwa für Personen, geografische Orte oder Institutionen, in einem Text zu identifizieren. Statt wie bislang jedem Wort ein Tag zuzuweisen, werden bestimmte Wörter bzw. *Sequenzen* von Wörtern als *Named Entity* ausgewiesen. Named Entity Recognition ermöglicht es, bestimmte Informationen aus großen, unstrukturierten Textmengen zu extrahieren und strukturiert, z.&nbsp;B. in Form einer Tabelle, zu speichern.

Für Named Entity Recognition können wir abermals `spacy` und `stanza` verwenden, wobei wir den `stanza_tagger` noch einmal "umprogrammieren" müssen:

In [38]:
#Der Tagger vor 'ner' ist eine Abhängigkeit desselben und muss entsprechend mitgeladen werden.
stanza_tagger = stanza.Pipeline(lang="de", processors="tokenize,ner")

2024-06-11 09:48:27 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-11 09:48:27 INFO: Downloaded file to /Users/Yannick/stanza_resources/resources.json


2024-06-11 09:48:27 WARNING: Language de package default expects mwt, which has been added


2024-06-11 09:48:28 INFO: Loading these models for language: de (German):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| ner       | germeval2014 |



2024-06-11 09:48:28 INFO: Using device: cpu


2024-06-11 09:48:28 INFO: Loading: tokenize


2024-06-11 09:48:28 INFO: Loading: mwt


2024-06-11 09:48:28 INFO: Loading: ner


2024-06-11 09:48:30 INFO: Done loading processors!


Nun können wir uns den in `sentence` gegebenen Satz im Hinblick auf Eigennamen taggen lassen, und zwar von beiden Taggern:

In [39]:
sentence = "Angela Merkel traf den Vorstand der Deutschen Bahn in Frankfurt am Main."

#Tagging
spacy_output = spacy_tagger(sentence)
stanza_output = stanza_tagger(sentence)

"""Ausgabe der beiden Outputs nebeneinander mithilfe einer 'range'-Schleife über 'spacy_out.ents' 
('ents'-Attribut schafft ein Objekt zur Iteration über getaggte Entitäten)"""
print(f"{'Wortform':30}{'spacy':10}{'stanza':10}\n{'-'*50}")
for i in range(len(spacy_output.ents)):
    """Zugriff auf Wortform ('text'-Attribut) der getaggten Entität, sowie das verliehene Tag
    vonseiten 'spacy' ('label_'-Attribut) bzw. 'stanza' ('type'-Attribut). Diese Art der Iteration
    und Ausgabe funktioniert natürlich nur so lange dieselben Entitäten getaggt wurden, was
    bei diesem kurzen Beispielsatz der Fall ist."""
    print(f"{spacy_output.ents[i].text:30}{spacy_output.ents[i].label_:10}{stanza_output.ents[i].type:10}")

Wortform                      spacy     stanza    
--------------------------------------------------
Angela Merkel                 PER       PER       
Deutschen Bahn                ORG       ORG       
Frankfurt am Main             LOC       LOC       


Das klappt wunderbar: Angela Merkel wurde als Person ("PER"), die Deutsche Bahn als Organisation ("ORG") und Frankfurt am Main als geografischer Ort ("LOC") erkannt. 



`spacy` unterscheidet fürs deutschsprachige NER-Tagging folgende vier Tags:

In [40]:
for label in spacy_tagger.get_pipe("ner").labels:
    print(f"{label:10}{spacy.explain(label)}")

LOC       Non-GPE locations, mountain ranges, bodies of water
MISC      Miscellaneous entities, e.g. events, nationalities, products or works of art
ORG       Companies, agencies, institutions, etc.
PER       Named person or family.


[`stanza`](https://stanfordnlp.github.io/stanza/ner_models.html) verwendet dieselben vier Tags.

`spacy` bietet auch hier die Möglichkeit der Visualisierung, einzig den Parameter `style` müssen wir anpassen:

In [41]:
displacy.render(spacy_output, style="ent", jupyter=True)

An der grafischen Ausgabe erkennt man gut, dass es sich nicht um eine Wort-für-Wort-Klassifikation wie bei allen anderen Taggingarten bisher handelt, sondern um eine *Sequence Labeling Task*. 

***

✏️ **Übung 6:** Lad Dir mithilfe der [API für Wikipedia](https://pypi.org/project/Wikipedia-API/) `wikipediaapi` (vgl. Zusatzübungen zum Notebook "Reguläre Ausdrücke") den *englischsprachigen* Artikel zu einer prominenten Person Deiner Wahl herunter und extrahier alle darin erwähnten Personen mithilfe von `spacy` und `stanza`. Welche Unterschiede zeigen sich beim Vergleich der zehn häufigsten Personen in den beiden Outputs? Verwend `pandas` für diesen Vergleich.

Lass Dir außerdem sämtliche von `spacy` gefundenen Named Entities visualisieren (also nicht nur die Personen). Erkennst Du einen Unterschied im Vergleich zum NER-Tagging deutschsprachiger Texte?

<details>
  <summary>💡 Tipp 1</summary>
  <br>Spezifiziere die richtige Sprache beim Abrufen des Artikels mithilfe von <code>wikipediaapi</code>.
</details>
<br>

<details>
  <summary>💡 Tipp 2</summary>
  <br>Da Du nun einen englischsprachigen Text taggst, musst Du die beiden Tagger neu programmieren, d.&nbsp;h. ihnen ein englischsprachiges Modell zugrunde legen. Im Falle von <code>spacy</code> lädst Du dafür z.&nbsp;B. das Modell "en_core_web_sm" herunter und übergibst es dem <code>load</code>-Befehl (<a href="https://spacy.io/models">hier</a> findest Du eine Übersicht aller zur Verfügung stehenden Modelle bei <code>spacy</code>). Bei <code>stanza</code> reicht die Spezifikation des <code>lang</code>-Parameter als "en" beim Initialisieren des Taggers.
</details>
<br>

<details>
  <summary>💡 Tipp 3</summary>
  <br>Die beiden Tagger verwenden dasselbe Tag für Personen, aber ist es auch das gleiche Tag wie beim Tagging deutschsprachiger Texte? 
</details>

In [42]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




Wunderbar. Schauen wir uns abschließend noch Sentiment Analysis an.

***

## Sentiment Analysis

Bei der Sentiment Analysis werden einzelne Sätze oder auch ganze Texte nach der in ihnen ausgedrückten Stimmung (engl. *sentiment*) beurteilt. Je nach Tagger wird dieses Sentiment über sprachliche Tags von "positiv" über "neutral" bis "negativ" oder über eine Zahl auf einer entsprechenden, numerischen Skala ausgedrückt (etwa von 1 bis -1).

Sentiment Analysis stellt eine sinnvolle Technik zur Operationalisierung bestimmter Fragestellungen dar, etwa wenn Kommentare in Sozialen Medien hinsichtlich besonders negativen Inhalten, in denen z.&nbsp;B. Hass ausgedrückt wird, (vor-)gefiltert werden sollen. Für weiterreichendere Aussagen über Texte greift die Methode oft zu kurz, zumal die ihr zugrundeliegende Skala nur eine Dimension kennt. Weiter haben Sentiment Tagger oft Mühe, mit sprachlichen Phänomenen wie mehrfacher Negation ("etwas nicht ungern tun") oder Ironie umzugehen.

Wollen wir aber die Probe auf's Exempel machen. Wir nutzen dazu [`germansentiment`](https://pypi.org/project/germansentiment/) sowie ein paar simple Sätze aus `sentences`. Die Sätze sind aufsteigend nach Negativität sortiert, wobei dies bis zu einem gewissen Grad natürlich eine subjektive Einschätzung darstellt:

In [43]:
from germansentiment import SentimentModel

sentiment_tagger = SentimentModel()

sentences = ["Ich bin ein riesengroßer Fan von Schokolade.",
             "Was ich auch durchaus mag, sind frisch gepflückte Himbeeren.",
             "Salz und Pfeffer dürfen nicht fehlen.",
             "Von Kartoffelauflauf halte ich nichts.",
             "Mit gekochten Möhren kannst Du mich jagen."]

sentiment_tagger.predict_sentiment(sentences)

/Users/Yannick/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


['positive', 'positive', 'neutral', 'negative', 'neutral']

Entsprechend der subjektiven Rangfolge, verleiht `germansentiment` den ersten beiden Sätzen das Tag "positive", dem mittleren Satz "neutral" sowie dem vierten Satz "negative". Einzig beim letzten Satz weicht das verliehene Tag vom subjektiven Urteil ab, dass es sich um einen (sehr) negativen Satz handelt. Auch hier gilt: Die erwähnten Tags wurden zum Zeitpunkt des Verfassens dieses Notebooks verliehen und können nach einer Aktualisierung der Sprachmodelle anders ausfallen (s.&nbsp;o.).

`germansentiment` vergibt insgesamt die drei Tags "positive", "neutral" und "negative", wobei es für jedes Tag separat errechnet, wie wahrscheinlich dieses beim gegebenen Satz zutrifft. Oben haben wir jeweils das wahrscheinlichste Tag zurückerhalten. Indem wir den Parameter `output_probabilities` aber auf `True` setzen, erhalten wir neben den wahrscheinlichsten Tags auch Auskunft darüber, wie wahrscheinlich jedes einzelne Tag war. 

Die beiden Rückgabewerte, die Tags und die Wahrscheinlichkeiten, weisen wir sinnvollerweise zwei verschiedenen Variablen zu. Anschließend iterieren wir über die einigermaßen verschachtelten Objekte, um uns Tags und Wahrscheinlichkeitsverteilungen pro Satz ausgeben zu lassen:

In [44]:
tags, probabilities = sentiment_tagger.predict_sentiment(sentences, output_probabilities=True)

for i in range(len(sentences)):
    print(f"Der Satz '{sentences[i]}' ist am wahrscheinlichsten {tags[i]}. Insgesamt sieht die Wahrscheinlichkeitsverteilung so aus:")
    
    for tag in probabilities[i]:
        print(f"\t{tag[0]} hatte eine Wahrscheinlichkeit von {tag[1]:.2f}")
    
    print("\n")

Der Satz 'Ich bin ein riesengroßer Fan von Schokolade.' ist am wahrscheinlichsten positive. Insgesamt sieht die Wahrscheinlichkeitsverteilung so aus:
	positive hatte eine Wahrscheinlichkeit von 0.68
	negative hatte eine Wahrscheinlichkeit von 0.19
	neutral hatte eine Wahrscheinlichkeit von 0.13


Der Satz 'Was ich auch durchaus mag, sind frisch gepflückte Himbeeren.' ist am wahrscheinlichsten positive. Insgesamt sieht die Wahrscheinlichkeitsverteilung so aus:
	positive hatte eine Wahrscheinlichkeit von 0.97
	negative hatte eine Wahrscheinlichkeit von 0.02
	neutral hatte eine Wahrscheinlichkeit von 0.00


Der Satz 'Salz und Pfeffer dürfen nicht fehlen.' ist am wahrscheinlichsten neutral. Insgesamt sieht die Wahrscheinlichkeitsverteilung so aus:
	positive hatte eine Wahrscheinlichkeit von 0.00
	negative hatte eine Wahrscheinlichkeit von 0.00
	neutral hatte eine Wahrscheinlichkeit von 1.00


Der Satz 'Von Kartoffelauflauf halte ich nichts.' ist am wahrscheinlichsten negative. Insgesamt si

Es zeigt sich, dass sich `germansentiment` bei seiner Einschätzung, dass der letzte Satz in `sentences` neutral ist, sogar sehr sicher ist. Vielleicht liegt dies daran, dass der Satz umgangssprachlich formuliert ist; vielleicht aber auch daran, dass der Tagger die vorangehenden Sätze nicht als Kontext mit einbezieht. Anders chatGPT, das analog zur subjektiven Einschätzung von oben urteilt:

<img src="../3_Dateien/Grafiken_und_Videos/sentiment_analysis_chatGPT.png">

OpenAI bietet übrigens auch eine API, mit der wir über Python Anfragen an chatGPT schicken können. Die Anzahl an Anfragen über diesen Kanal ist allerdings stark eingeschränkt und nur zahlende Nutzer:innen können die API sinnvoll einsetzen. 

Üben wir Sentiment Analysis!

***

✏️ **Übung 7:** Find heraus, ob es sich beim Koalitionsvertrag von 2021 um einen positiven, neutralen oder negativen Text handelt. Überleg Dir genau, was für Input `predict_sentiment` erwartet, d.&nbsp;h. wie Du den Koalitionsvertrag sinnvollerweise taggst.

<details>
  <summary>💡 Tipp 1</summary>
  <br><code>predict_sentiment</code> erwartet eine Liste mit strings. Zwar könntest Du den gesamten Koalitionsvertrag als ein string auf einer Liste übergeben, angesichts der Länge des Texts ist es aber sinnvoller, diesen erst in Sätze zu splitten (u.&nbsp;a. weil Du nur so den Fortschritt des Taggings verfolgen kannst, s. Tipp 2). <code>nltk</code> bietet übrigens auch dafür eine Funktion, nämlich <code>sent_tokenize</code> (vgl. <code>word_tokenize</code> oben). Nun übergibst Du Satz für Satz an <code>germansentiment</code>, allerdings trotzdem als Element einer Liste, denn das ist das erwartete Inputformat. Abschließend berechnest Du, welches Tag am häufigsten verliehen wurde.
</details>
<br>
<details>
  <summary>💡 Tipp 2</summary>
    <br>Das Tagging dauert lange. Bau mithilfe von <code>tqdm</code> eine Fortschrittsanzeige ein, um den Prozess im Blick behalten zu können (vgl. Zusatzübungen zum Notebook "Web Scraping"). Führ den Code gerne auch über Nacht aus und stell sicher, dass sich Dein Rechner nicht nach einer gewissen Zeit von alleine ausschaltet. 
    <br><br>Weiter empfiehlt es sich, den Tagging-Code zunächst bei einigen wenigen Sätzen auszuprobieren. Ansonsten musst Du jeweils sehr lange warten, bis der Code ausgeführt wurde und Du überprüfen kannst, ob das Resultat wie gewünscht aussieht.
</details>
<br>

<details><summary>📌 Herausforderung </summary>
<br>Find zusätzlich heraus, bei welchen Sätzen des Koalitionsvertrags ein negatives bzw. positives Sentiment mit einer Wahrscheinlichkeit von über 50% getaggt wurde.
</details>

In [45]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Bevor wir uns dem Anwendungsfall widmen, soll noch auf eine hilfreiche Ressource für Sentiment Analysis hingewiesen werden: [SentiWS](https://wortschatz.uni-leipzig.de/de/download). Das Projekt Deutscher Wortschatz stellt mit SentiWS eine umfangreiche Liste von Wörtern zur Verfügung, die typischerweise Sentiment ausdrücken. Jedes Wort wird mit einer Zahl zwischen 1 und -1 bewertet. Anstatt einen fertigen Tagger wie `germansentiment` über Deine Daten laufen zu lassen, kannst Du auch das "Sentiment-Wörterbuch" von SentiWS benutzen. Dadurch ermittelst Du nicht nur, welches Sentiment Dein Text ausdrückt, sondern erfährst zudem, aufgrund welcher Wörter dieses Sentiment zustandekommt.

***

## 🔧 Anwendungsfall: (D)ein Korpus taggen 🏷️

Setz nun die erlernten Taggingkenntnisse um und reicher eigene Daten mit Zusatzinformationen an. Schau Dir ebenfalls den Musteranwendungsfall in den Lösungen an.

***

Gute Arbeit!

<br><br>
***
<table>
      <tr>
        <td>
            <img src="../3_Dateien/Lizenz/CC-BY-SA.png" width="400">
        </td> 
        <td>
            <p>Dieses Notebook sowie sämtliche weiteren <a href="https://github.com/yannickfrommherz/exdimed-student/tree/main">Materialien zum Programmierenlernen für Geistes- und Sozialwissenschaftler:innen</a> sind im Rahmen des Projekts <i>Experimentierraum Digitale Medienkompetenz</i> als Teil von <a href="https://tu-dresden.de/gsw/virtuos/">virTUos</a> entstanden. Erstellt wurden sie von Yannick Frommherz unter Mitarbeit von Anne Josephine Matz. Sie stehen als Open Educational Resource nach <a href="https://creativecommons.org/licenses/by-sa/4.0/">CC BY SA</a> zur freien Verfügung. Für Feedback und bei Fragen nutz bitte das <a href="https://forms.gle/VsYJgy4bZTSqKioA7">Kontaktformular</a>.
        </td>
      </tr>
</table>